In [324]:
#!/usr/bin/env python3

from rdflib import Graph, Namespace, URIRef, BNode, Literal, RDF
from rdflib.namespace import SKOS, XSD, OWL, DC
from rdflib.namespace import DCTERMS as DCT

from pymarc import Record, Field, XMLWriter, MARCReader

import sys
import logging
import datetime
import subprocess
import urllib
from collections import namedtuple, Sequence

CONVERSION_PROCESS = "YSO-SKOS to MARC 0.3"
CONVERSION_URI = "https://www.kiwi.fi/x/XoK6B" # konversio-APIn uri tai muu dokumentti, jossa kuvataan konversio
CREATOR_AGENCY = "FI-NL" # Tietueen luoja/omistaja & luetteloiva organisaatio, 003 & 040 kentat

# vuosi, kuukausi, päivä
DEPREKOINTIRAJA = datetime.date(2018, 5, 1)

ENDPOINT_ADDRESS = "http://api.dev.finto.fi/sparql"
# initialize logging
logformat = '%(levelname)s: %(message)s'
loglevel = logging.DEBUG
logging.basicConfig(format=logformat, level=loglevel)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

#SKOS=Namespace('http://www.w3.org/2004/02/skos/core#')
YSO=Namespace('http://www.yso.fi/onto/yso/')
YSOMETA=Namespace('http://www.yso.fi/onto/yso-meta/')
YSA=Namespace('http://www.yso.fi/onto/ysa/')
YSAMETA=Namespace('http://www.yso.fi/onto/ysa-meta/')
ALLARS=Namespace('http://www.yso.fi/onto/allars/')
ALLARSMETA=Namespace("http://www.yso.fi/onto/allars-meta/")
KOKO=Namespace('http://www.yso.fi/onto/koko/')
LCSH = Namespace("http://id.loc.gov/authorities/subjects/")
RDAU=Namespace('http://rdaregistry.info/Elements/u/')
ISOTHES=Namespace('http://purl.org/iso25964/skos-thes#')
SKOSEXT=Namespace('http://purl.org/finnonto/schema/skosext#')

VOCABCODE = 'yso'

LANGUAGES = {
    'fi': 'fin',
    'sv': 'swe',
    'en': 'eng',
    'de': 'ger',
    'et': 'est',
    'fr': 'fre',
    'it': 'ita',
    'ru': 'rus',
#    'se': 'sme', # pohjoissaami
    'sme': 'sme', # pohjoissaami
    'sma': 'sma', # etalasaami
    'smn': 'smn', # inarinsaami,
    'sms': 'sms', # koltansaami,
    'smj': 'smj', # luulajansaami
}



TRANSLATIONS = {
    SKOSEXT.partOf: {
        "fi": "osa kokonaisuutta/käsitettä",
        "sv": "är en del av",
        "en": "is part of"
    },
    "682iDEFAULT": {
        "fi": "Käytöstä poistettu termi.",
        "sv": "Avlagd term.",
        "en": "This term is obsolete."
    },
    "688aCREATED": {
        "fi": "Luotu",
        "sv": "Skapad",
        "en": "Created"
    },
    "688aMODIFIED": {
        "fi": "Viimeksi muokattu",
        "sv": "Senast editerad",
        "en": "Last modified"
    }
}

# arvot tulevat osakentan $w 1. merkkipaikkaan
SEEALSOPROPS = {
    SKOS.broader : 'g',
    SKOS.narrower : 'h',
    SKOS.related : 'n',
    RDAU.P60683 : 'a',
    RDAU.P60686 : 'b',
    SKOSEXT.partOf : 'i',
    ISOTHES.broaderPartitive : "g",
    ISOTHES.narrowerPartitive : "h"
}

#katso-viittauksen kentän tyyppi - selite
TERMGROUP = {
    SKOS.altLabel: {
        "fi": "ohjaustermi",
        "sv": "hänvisningsterm",
        "en": "entry term"
    },
    SKOS.hiddenLabel: {
        "fi": "piilotermi",
        "sv": "dold term",
        "en": "hidden term"
    },
    YSOMETA.singularPrefLabel: {
        "fi": "käytettävän termin yksikkömuoto",
        "sv": "föredragen term i singular",
        "en": "singular entry term"
    },
    YSOMETA.singularAltLabel: {
        "fi": "ohjaustermin yksikkömuoto",
        "sv": "hänvisningsterm term i singular",
        "en": "singular entry term"
    }
}

# paikka 5, 'n' = uusi, 'c' = muuttunut/korjattu, d = poistettu
LEADERNEW = '00000nz  a2200000n  4500'
LEADERCHANGED = '00000cz  a2200000n  4500'
LEADERDELETED = '00000dz  a2200000n  4500'
# paikka 10 pitaisiko olla 'z' = muut luettelointisaannot?
# paikka 17, pitaisiko taydentaa asialisamaareen tyyppi
# (aihe, aika, paikka, kieli, soveltumaton)
# paikka 29, pitaako koodata?
CODESKUVAILU = '|n|azz||babn          || ana    ||'
CODESEIKUVAILU = '|n|ezz||bbbn          || ana    ||'

GROUPSEIKUVAILU = [ISOTHES.ConceptGroup, ISOTHES.ThesaurusArray,
            YSOMETA.Hierarchy, SKOS.Collection]

In [346]:
sys.argv = ['yso2marc', 
          '../../vocabularies/yso/yso-skos.ttl',
            "yso",
           "fi",
            ]
# TODO: mikä tiedosto ladattiin (parametrina), yso, yso-paikat, seko, genre

In [279]:
g = Graph()
g.parse(sys.argv[1], format='turtle')

In [338]:
from SPARQLWrapper import SPARQLWrapper

sparql = SPARQLWrapper(ENDPOINT_ADDRESS)
sparql.setQuery("""
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX yso-paikat: <http://www.yso.fi/onto/yso-paikat/>
PREFIX ysa: <http://www.yso.fi/onto/ysa/>
PREFIX ysameta: <http://www.yso.fi/onto/ysa-meta/>
PREFIX allars: <http://www.yso.fi/onto/allars/>
PREFIX allarsmeta: <http://www.yso.fi/onto/allars-meta/>
CONSTRUCT {
    ?concept skos:prefLabel ?prefLabel .
    ?concept skos:inScheme ?inScheme .
    ?concept owl:deprecated ?deprecated .
    ?concept a ?types .
}
FROM yso-paikat:
FROM ysa:
FROM allars:
WHERE {
  ?concept a skos:Concept .
  ?concept skos:prefLabel ?prefLabel .
  ?concept a ?types .
  
  OPTIONAL {?concept skos:inScheme ?inScheme .}
  OPTIONAL {?concept owl:deprecated ?deprecated .}
}
""")
# TODO: kielifiltteröinti (toteutus tod. näk. helpompi graafin läpikäynnissä)
# TODO: muokkaa kyselyä sen mukaan, mikä tiedosto luetaan - FROM-lauseet
# TODO: LCSH mukaan g2-graafiin
# TODO: 4 eri tiedostoa; yso, yso-paikat, seko, genre
sparql.setMethod("GET")
g2 = sparql.query().convert()


In [339]:
logging.debug(len(g))

logging.debug(len(g2))


In [350]:
#g2.parse("lcsh-preflabels-2018-03-26.nt", format="turtle")
# TODO: lisätään lcsh-tiedostoon marc-tiedot
# (joko ottamalla ne mads/rdf-tiedostosta tai lukemalla nämä suoraan MARCXML-tiedostoa)
# Haaste: pitää pystyä asettaa ne samalla tavoin kuin LCSH on itse tehnyt (alakentät jne.)

In [347]:
# MAIN

def main():
    if len(sys.argv) < 4:
        logging.error("Usage: python3 yso2marc2.py file outputfilename language generateAllLanguages=False")
        return
    PRIMARYLANG = sys.argv[3]
    convert_others = sys.argv[4] if len(sys.argv) > 4 else False
    if convert_others:
        for lang in LANGUAGES:
            convert(lang, sys.argv[2])
    else:
        convert(PRIMARYLANG, sys.argv[2])
    
if __name__ == "__main__":
    main()
    


In [318]:
# apufunktiot

ValueProp = namedtuple("ValueProp", ['value', 'prop'])

def getPreflabel(target, language, prop, concept):
    label = ""
    success = False
    labels = g.preferredLabel(target, lang=language)
    try:
        label = labels[0][1]
        success = True
    except IndexError:
        print("WARNING: Couldn't find preflabel for target %s in language: %s. Skipping property %s target for concept %s." %
              (target, language, prop, concept), sys.stderr)
    return (success, label)


def getValues(target, props, language=None, datatype=None, graph=g):
    """Given a subject, get all values for a list of properties.

    Args:
        target (URIRef|BNode): Concept.
        props (URIRef|sequence(URIRef)): Property or list of properties to search for.
        language (str, optional): Language of literals. Defaults to None (return all literals with languages).
            Set to empty string ("") for empty lang tag.
        datatype (URIRef, optional): Datatype of datatyped literals. Defaults to None (return all literals with datatypes).
        graph (Graph, optional): The graph from which to search the properties for.
            Defaults to 'g' in which the main ttl file is loaded.

    Returns:
        list(TypeValue): List containing TypeValue namedtuples
            
            prop (URIRef): Matched property
            value (URIRef|BNode|Literal): For matched property, object value

    Raises:
        ValueError: If parameters do not respect the required types

    """
    if isinstance(props, URIRef):
        # cast to list in order to uniform code
        props = [props]
    
    if not (isinstance(target, URIRef) or isinstance(target, BNode)):
        raise ValueError("Parameter 'target' must be of type URIRef or BNode.")
    elif isinstance(props, str) or not isinstance(props, Sequence):
        raise ValueError(
            "Type of parameter 'props' must be a URIRef or sequence; got %s." % (type(props)))
    elif language is not None and not isinstance(language, str):
        raise ValueError("Parameter 'language' must be string if set.")
    elif datatype is not None and not isinstance(datatype, URIRef):
        raise ValueError("Parameter 'datatype' must be URIRef if set.") 
    
    v = []
    
    # setup the language filtering
    if language is not None:
        if language == '':  # we only want not language-tagged literals
            langfilter = lambda l: l.language is None
        else:
            langfilter = lambda l: l.language == language
    else:  # we don't care about language tags
        langfilter = lambda l: True
    
    # setup the datatype filtering
    if datatype is not None:
        typefilter = lambda l: l.datatype == datatype
    else:
        typefilter = lambda l: True
    
    
    
    for prop in props:
        if not isinstance(prop, URIRef):
            raise ValueError(
            "Types of properties must be URIRefs; got %s from property '%s'." % (type(prop), str(prop)))
        
        # values that pass restrictions are returned
        values = [l for l in graph.objects(target, prop) if 
                  isinstance(l, URIRef) or isinstance(l, BNode) or 
                  (langfilter(l) and l.datatype == None)  or 
                  (typefilter(l) and l.language == None)]
        
        # loop through the values and add them to the list
        for val in values:
            v.append(ValueProp(value=val, prop=prop))
    return v

def getInvValues(target, props, graph=g):
    #TODO: todo
    return

# apufunktio urlien parsimiseen merkkijonosta
# mietittävä uudelleen, jos näitä rakenteistetaan
def getURLs(string):
    urls = []
    for word in string:
        if len(word) < 10:
            continue
        if word[0] in ["(", "["]:
            word = word[1:-1]
        res = urllib.parse.urlparse(word)
        if res.scheme in ("http", "https") and \
            len(res.netloc) > 3 and "." in res.netloc:
            urls.append(word)
    return urls

# TODO: language support!
# Currently supports only 1-language vocabularies
def getPrefLabel(valuePropObj, graph=g):
    if valuePropObj.prop == SKOS.prefLabel:
        return str(valuePropObj.value)
    else:
        matchURIRef = URIRef(valuePropObj.value)
        prefLabel = None
        
        for type2, prefLabel in sorted(graph.preferredLabel(matchURIRef,
                                    labelProperties=(SKOS.prefLabel,))):
            return str(prefLabel)
        return str(valuePropObj.value)

In [349]:
def convert(language, outputfilename, ignoreOtherGraphs=True, keepDeprecated=False):
    # keepDeprecated: False tarkoittaa, että EI haluta, ja TRUE, että halutaan
    # vanhennetut käsitteet lopputulokseen
    print("Processing '%s' language" % (language))
    incrementor = 0
    writer = XMLWriter(open(outputfilename + "MARC-" + language + ".mrcx", 'wb'))
    # for concept (a skos:Concept) in graph
    for concept in sorted(g.subjects(RDF.type, SKOS.Concept)):
        incrementor += 1
        if incrementor % 1000 == 0:
            print("Processing %sth concept" % incrementor)
        # if deprecated, skip
        if not keepDeprecated and (concept, OWL.deprecated, Literal(True)) in g:
            continue
        
        rec = Record()   
        
        # Organisaation ISIL-tunniste -> 003
        rec.add_field(
            Field(
                tag='003',
                data = CREATOR_AGENCY
            )
        )
        # dct:modified -> 005 EI TULOSTETA, 688 
        # tutkitaan, onko käsite muuttunut vai alkuperäinen
        # ja valitaan leader sen perusteella
        mod = g.value(concept, DCT.modified, None)
        if mod is None:
            rec.leader = LEADERNEW
        else:
            rec.leader = LEADERCHANGED
            modified = mod.toPython() # datetime.date or datetime.datetime object
            if not keepDeprecated and (concept, OWL.deprecated, Literal(True)) in g and DEPRECATEDRAJA > modified:
                continue # skipataan ennen vanhentamisrajaa vanhennetut termit
            
        # dct:created -> 008
        crt = g.value(concept, DCT.created, None) # DCT.created?
        if crt is None:
            created = datetime.date(1980, 1, 1)
        else:
            created = crt.toPython() # datetime.date or datetime.datetime object
        
        code = CODESKUVAILU
        
        for conceptType in g.objects(concept, RDF.type):
            if conceptType in GROUPSEIKUVAILU:
                code = CODESEIKUVAILU
                continue
        if (concept, OWL.deprecated, Literal(True)) in g:
            rec.leader = LEADERDELETED
            code = CODESEIKUVAILU
        
        try:
            rec.add_field(
                Field(
                    tag='008',
                    data=created.strftime('%y%m%d') + code
                )
            )
        except AttributeError:
            print("WARNING: could not convert date to another format: Concept %s, value: %s" %
                 (concept, created))
        
        # 024 muut standarditunnukset - käsitteen URI tallennetaan tähän
        rec.add_field(
            Field(
                tag='024',
                indicators = ['7', ' '],
                subfields = [
                    'a', concept,
                    '2', "uri"
                ]
            )
        )
        
        # 034 paikkojen koordinaatit - yso-paikat?
        # 035 yso-tietueen numero?
        
        # 040 luetteloiva organisaatio
        # TODO: tarkista miten vocabcode merkitään muille kuin YSOlle, muista myös muut kentät
        # Jarmo: ei kielikoodia muille sanastoille?
        rec.add_field(
            Field(
                tag='040',
                indicators = [' ', ' '],
                subfields = [
                    'a', CREATOR_AGENCY,
                    'b', LANGUAGES[language],
                    'e', "rda",
                    'f', VOCABCODE + "/" + LANGUAGES[language]
                ]
            )
        )
        # 043 - ysopaikat, kaytetaanko
        # http://marc21.kansalliskirjasto.fi/aukt/01X-09X.htm#043
        
        # 045 - yso-ajanjaksot, kaytetaanko
        # http://marc21.kansalliskirjasto.fi/aukt/01X-09X.htm#045
        
        # 046 - erikoiskoodatut ajanjaksot? 
        
        # 052 - maantieteellinen luokitus
        # 7#$a(480)$2udc$0http://udcdata.info/004604
        # jos 151 kaytossa, pitaisiko kayttaa? Jarmo: UDC-luokitus, Suomi "(480)"
        
        #ConceptGroup / skos:member -> 065 yso-aihealuekoodi
        # vain siina tapauksessa, kun ne halutaan mukaan Asteriin
        # jos luokkanumeroa ei löydy, ei tulosteta
        # TODO: vain jos VOCABCODE = "yso", tehdään tämä
        for group in sorted(g.subjects(SKOS.member, concept)):
            if not keepDeprecated and (group, OWL.deprecated, Literal(True)) in g:
                continue # skip deprecated group concepts
            if (group, RDF.type, ISOTHES.ConceptGroup) not in g:
                #print("Not ISOTHES.ConceptGroup", concept)
                continue
            # group code: first try using skos:notation, otherwise extract from label
            groupno = g.value(group, SKOS.notation, None)
            if groupno is None:
                prefLabels = sorted(g.preferredLabel(group, lang=language,
                                                     labelProperties=(SKOS.prefLabel,)))
                if len(prefLabels) == 0: 
                    print("WARNING: Couldn't find preflabel for target %s in language: %s. Skipping property %s target for concept %s." %
                      (group, language, SKOS.member, concept), sys.stderr)
                    continue
                elif len(prefLabels) != 1:
                    print("WARNING: Multiple prefLabels detected for concept %s in language %s. Taking the first only." %
                      (concept, language), sys.stderr) 
                groupname = str(prefLabels[0][1])
                try:
                    groupno = groupname[0:groupname.index(" ")]
                    groupname = groupname[len(groupno) + 1:]
                except ValueError:
                    print("WARNING: Tried to parse group number for group %s from preflabel %s in language %s but failed." %
                      (group, language), sys.stderr)
                    continue

            rec.add_field(
                Field(
                       tag='065',
                       indicators = [' ', ' '],
                       subfields = [
                           'a', groupno,
                           'c', groupname,
                           '0', group,
                           '2', VOCABCODE
                       ]
                )
            )
        
        # 080 - UDK-luokka. Asiasanaan liittyva UDK-luokka
        
        # 147 Tapahtuman nimi. Ei kayteta?
        
        # 148 Aikaa merkitseva termi. Selvitetaan.
        
        # skos:prefLabel -> 150 aihetta ilmaiseva termi
        prefLabels = sorted(g.preferredLabel(concept, lang=language, labelProperties=(SKOS.prefLabel,)))
        if len(prefLabels) == 0:
            print("WARNING: Couldn't find preflabel for concept %s in language %s. Skipping the whole concept." %
              (concept, language), sys.stderr)
            continue;
        elif len(prefLabels) != 1:
            print("WARNING: Multiple prefLabels detected for concept %s in language %s. Choosing the first." %
                  (concept, language), sys.stderr) 
        #for prefLabel in prefLabels:
        # TODO: tunnista kasitteen tyyppi, aika, yleinen, paikka, genre
        # -> 148, 150, 151, 155, 162
        tag = "150"
        if (concept, SKOS.inScheme, YSO.places) in g:
            tag = "151"

        rec.add_field(
            Field(
                tag=tag,
                indicators = [' ', ' '],
                subfields=[
                            'a',str(prefLabels[0][1])
                          ]
            )
        )
        
        # skos:altLabel -> 447, 448, 450, 451, 455
        #450 katso-viittaus
        # poistetaan toisteiset skos:hiddenLabelit
        seen_values = set()
        for valueProp in sorted(getValues(concept, [SKOS.altLabel, YSOMETA.singularPrefLabel,
                                                YSOMETA.singularAltLabel, SKOS.hiddenLabel], language=language),
                                key=lambda o: str(o.value)): 
            #print("meneeko tänne")
            if valueProp.prop == SKOS.hiddenLabel:
                if str(valueProp.value) in seen_values:
                    continue
            seen_values.add(str(valueProp.value))
            
            tag = "450"
            if (concept, SKOS.inScheme, YSO.places) in g:
                tag = "451"
            rec.add_field(
                Field(
                    tag = tag,
                    indicators = [' ', ' '],
                    subfields = [
                        'a', str(valueProp.value),
                        'i', TERMGROUP[valueProp.prop][language]
                    ]
                )
            )
        
        # broader/narrower/related/successor/predecessor/skosext:partOf
        # -> 550 "katso myos" viittaus
        # HUOM: Objektit vain olioita
        # TODO: ysoon lisätään myöhemmin partOf-suhteiden käänteinen suhde
        # TODO: useat erityyppiset i-kentät eivät toimi tällä hetkellä
        for prop, wval in SEEALSOPROPS.items():
            for target in sorted(g.objects(concept, prop)):
                if not keepDeprecated and (target, OWL.deprecated, Literal(True)) in g:
                    continue # skip deprecated concepts
                
                (success, label) = getPreflabel(target, language, prop, concept)
                if not success:
                    continue # skip concepts with no preferred label
                
                tag = "550" # alustetaan 550-arvoon
                if (target, SKOS.inScheme, YSO.places) in g:
                        tag = "551"
                        
                if wval == "i":
                    if (target, SKOS.inScheme, YSO.places) in g:
                        if prop == SKOSEXT.partOf:
                            subfields = ['w', 'g']
                        elif prop == SKOSEXT.hasPart:
                            subfields = ['w', 'h']
                        else:
                            subfields = ['w', wval,
                                     "i", TRANSLATIONS[prop][language]
                                    ]
                    else:
                        subfields = ['w', wval,
                                     "i", TRANSLATIONS[prop][language]
                                    ]
                else:
                    subfields = ['w', wval]
                        
                subfields.append('a')
                subfields.append(str(label))
                subfields.append('0')
                subfields.append(target)
                
                rec.add_field(
                    Field(
                        tag = tag,
                        indicators = [' ', ' '],
                        subfields = subfields
                    )
                )    
        
        # dc:source -> 670 kasitteen tai kuvauksen lahde
        # mikäli kielikoodilla ei ole propertille arvoa, ohjelma ei tulosta tätä kenttää
        # voidaanko tunnistaa, onko lahteessa URI, jolloin
        # $u-osakenttaan laitetaan URI
        # 4.5.2018 - palataan myöhemmin tähän
        for source in sorted(
            g.preferredLabel(concept, lang=language,
            labelProperties=(DC.source,))
        ):
            subfields = [
                'v', str(source[1])
            ]
            # TODO: linkkien koodaus tarkistetaan/tehdään myöhemmin
            #urls = getURLs(source[1])
            #for url in urls:
            #    subfields.append("u")
            #    subfields.append(url)
                
            rec.add_field(
                Field(
                    tag='670',
                    indicators = [' ', ' '],
                    subfields = subfields
                )
            )
        # skos:definition -> 677 huomautus maaritelmasta
        # maaritelman lahde voidaan merkita osakenttaan $v
        # sita varten tulee sopia tavasta merkita tama lahde, jotta
        # se voidaan koneellisesti erottaa tekstista
        # JS ehdottaa: jos tekstissa on merkkijono ". Lahde: ",
        # kaikki sen perassa oleva teksti merkitaan osakenttaan $v
        # enta jos linkki lahteen perassa?
        # JS ehdottaa: linkki aivan viimeisena sanana
        # 4.5.2018 - palataan myöhemmin tähän
        for definition in sorted(
            g.preferredLabel(concept, lang=language,
            labelProperties=(SKOS.definition,))
        ):
            subfields = [
                'a', str(definition[1])
            ]
            # TODO: linkkien koodaus tarkistetaan/tehdään myöhemmin
            #urls = getURLs(definition[1])
            #for url in urls:
            #    subfields.append("u")
            #    subfields.append(url)
                
            rec.add_field(
                Field(
                    tag='677',
                    indicators = [' ', ' '],
                    subfields = subfields
                )
            )
        
        # skos:note -> 680 yleinen huomautus, julkinen
        for valueProp in sorted(getValues(concept, [SKOS.note, SKOS.scopeNote, SKOS.example], language=language),
                                key=lambda o: str(o.value)):
            rec.add_field(
                Field(
                    tag='680',
                    indicators = [' ', ' '],
                    subfields = [
                        'i', str(valueProp.value)
                    ]
                )
            )
            
        # owl:deprecated -> 682 Huomautus poistetusta otsikkomuodosta
        # Ohjaus uuteen/uusiin käsitteisiin
        # seuraaja-suhde
        # a-kenttään seuraajan preflabel, 0-kenttään URI, i selite
        # TODO: onko seuraajaa vai ei, lisäksi mietittävä deprekoidun käsitteen
        # tyyppi (onko hierarkia jne.). Deprekaattorin huomautustekstiä kehitettävä
        # (kentät mietittävä uudelleen - EI skos:scopeNote kuten nyt on 4.5.2018)
        #if not keepDeprecated and (concept, OWL.deprecated, Literal(True)) in g:
        #    target = None
        #    for target in sorted(g.objects(concept, DCT.isReplacedBy)):
        #        if not keepDeprecated and (target, OWL.deprecated, Literal(True)) in g:
        #            continue # skip deprecated concepts
        #        
        #        (success, label) = getPreflabel(target, language, DCT.isReplacedBy, concept)
        #        if not success:
        #            continue # skip concepts with no preferred label
            
            rec.add_field(
                Field(
                    tag = '682',
                    indicators = [' ', ' '],
                    subfields = [
                        'a',  TRANSLATIONS["688aCREATED"][language] + ": " + created.strftime('%Y-%m-%d')
                    ]
                )
            )
        
        if created:
            rec.add_field(
                Field(
                    tag = '688',
                    indicators = [' ', ' '],
                    subfields = [
                        'a',  TRANSLATIONS["688aCREATED"][language] + ": " + created.strftime('%Y-%m-%d')
                    ]
                )
            )
        
        if mod and modified:
            rec.add_field(
                Field(
                    tag = '688',
                    indicators = [' ', ' '],
                    subfields = [
                        'a', TRANSLATIONS["688aMODIFIED"][language] + ": " + modified.strftime('%Y-%m-%d')
                    ]
                )
            )
            try:
                if type(modified) == datetime.datetime:
                    if created > modified.date():
                        print("WARNING: Created date later than modified for concept %s" % concept)
                else:
                    if created > modified:
                        print("WARNING: Created date later than modified for concept %s" % concept)
            except Exception:
                print("Date comparison failed for concept %s", concept)

                        
        # all skos:match*es -> 747-751 linkkikentta
        # halutaan linkit kaikkiin kieliversioihin
        # YSOn erikieliset preflabelit menevat 450-451 kenttiin
        # graafit haetaan etukateen ohjelman muistiin ohjelman alussa
        # haetaan api.dev.finto.fi/sparql -endpointista
        # tarvittavat graafit: ysa, allars, yso, yso-paikat, lcsh
        # miten LCSH haetaan? Yli 400 000 prefLabelia
        # tarjotaan talla hetkella vain api.skosmos.dev.finto.fi:n kautta
        # jolle ei paase ulkoa kasin ollenkaan.
        # 750 $a label, $4 relaatiotyyppi, $2 sanastolahde, $0 uri
        # miten $w? JS: ei oteta mukaan ollenkaan
        # 2.5.2018-kokouksessa päätettiin, että DCT.spatialia ei käännetä
        # MARC-muotoon
        valueProps = getValues(concept, [SKOS.prefLabel, SKOS.exactMatch, SKOS.closeMatch,
                                 SKOS.broadMatch, SKOS.narrowMatch, 
                                 SKOS.relatedMatch])
        sortedValueProps = sorted(valueProps, key=lambda o: getPrefLabel(o, graph=g2))
        for valueProp in sortedValueProps:
                
            if valueProp.prop == SKOS.prefLabel:
                # suodatetaan samankieliset, jotka menivät jo 1xx-kenttiin
                # valueProp.value sisältää tässä poikkeuksellisesti halutun literaalin
                if valueProp.value.language == language:
                    continue
                matchURIRef = URIRef(concept)
            else:
                # tehdään osumasta URIRef 
                matchURIRef = URIRef(valueProp.value)
                if not keepDeprecated and (matchURIRef, OWL.deprecated, Literal(True)) in g2:
                    continue # skip deprecated matches

            second_indicator = "7"
            tag = "750"
            if (matchURIRef, SKOS.inScheme, YSO.places) in g2: #or matchType == DCT.spatial:
                tag = "751"
            # TODO: nimetyt graafit, kohdista kyselyt niihin?
            # if we want to direct queries to spesific graphs, one per vocab, that graph
            # needs to be selected here based on the void:uriSpace
            
            if matchURIRef.startswith(LCSH):
                #second_indicator = "0"
                #sub2 = "lcsh"
                continue # skip LCSH concepts
            elif matchURIRef.startswith(ALLARS):
                if (matchURIRef, RDF.type, ALLARSMETA.GeographicalConcept) in g2: #or matchType == DCT.spatial:
                    tag = "751"
                sub2 = "allars"
            elif matchURIRef.startswith(KOKO):
                continue # skip KOKO concepts
            elif matchURIRef.startswith(YSA):
                if (matchURIRef, RDF.type, YSAMETA.GeographicalConcept) in g2: #or matchType == DCT.spatial:
                    tag = "751"
                sub2 = "ysa"
            elif matchURIRef.startswith(YSO):
                sub2 = "yso"
            else:
                second_indicator = "4"
                print("WARNING: Matched target %s did not belong to any known vocabulary", matchURIRef)
                # do not put subfield 2 in this case
            sub4 = ""
            if valueProp.prop == SKOS.broadMatch:
                sub4 = "BM"
            elif valueProp.prop == SKOS.narrowMatch:
                sub4 = "NM"
            elif valueProp.prop == SKOS.exactMatch:
                sub4 = "EQ"
            elif valueProp.prop == SKOS.prefLabel:
                sub4 = "EQ"
                if sub2 == "yso":
                    sub2 = sub2 + "/" + LANGUAGES[valueProp.value.language]
                rec.add_field(
                    Field(
                        tag=tag,
                        indicators = [' ', second_indicator],
                        subfields = [
                            'a', str(valueProp.value),
                            '4', sub4,
                            '2', sub2,
                            '0', concept
                        ]
                    )
                )
                continue
            elif valueProp.prop == SKOS.closeMatch:
                sub4 = "~EQ"
            else:
                sub4 = "RM"

            prefLabel = None
            for type2, prefLabel in sorted(g2.preferredLabel(matchURIRef,
                                    labelProperties=(SKOS.prefLabel,))):
                
                #TODO: Warn if many prefLabels for same language
                subfields = [
                    'a', str(prefLabel),
                    '4', sub4
                ]
                
                if second_indicator != "4":
                    subfields.append("2")
                    subfields.append(sub2)
                subfields.append("0")
                subfields.append(str(matchURIRef))
                #subfields.append("9")
                #try:
                #    subfields.append(LANGUAGES[prefLabel.language])
                #except KeyError:
                #    print("WARNING: LANGUAGES dictionary has no key for language '%s' found from %s skos:prefLabel. Skipping." %
                #      (prefLabel.language, matchURIRef), sys.stderr)
                #    continue

                rec.add_field(
                    Field(
                        tag=tag,
                        indicators = [' ', second_indicator],
                        subfields = [
                            'a', str(prefLabel),
                            '4', sub4,
                            '2', sub2,
                            '0', str(matchURIRef)
                            #'9', LANGUAGES[prefLabel.language]
                        ]
                    )
                )
                
            
            if not prefLabel and not ignoreOtherGraphs: 
                print("WARNING: Couldn't find preflabel for target %s. Skipping property %s target for concept %s." %
                  (str(matchURIRef), str(valueProp.prop), concept), sys.stderr)
                continue
        
        # Konversion tiedot -> 884
        tag = "884"
        rec.add_field(
                        Field(
                            tag=tag,
                            indicators = [' ', " "],
                            subfields = [
                                'a', CONVERSION_PROCESS,
                                'u', CONVERSION_URI
                            ]
                        )
                    )
        writer.write(rec)

    writer.close()
    #processCatmandu(language)
    

def processCatmandu(language="fi", vocabulary="yso"):
    print("catmandu convert MARC --type XML to MARC --type XML <" + vocabulary + "MARC-" + language + 
        ".mrcx | xmllint --format - >" + vocabulary + "MARC-" + language + "-formatted.mrcx")
    subprocess.run(
        "catmandu convert MARC --type XML to MARC --type XML <" + vocabulary + "MARC-" + language + 
        ".mrcx | xmllint --format - >" + vocabulary + "MARC-" + language + "-formatted.mrcx",
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True
    )

In [348]:
# aja generoidaksesi formatoitu versio tietynkielisesta filusta
kieli = "fi"
vocabulary = "yso"
processCatmandu(kieli, vocabulary)